<a href="https://colab.research.google.com/github/HNR1/Bachelors-project/blob/main/diffusers_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers diffusers tomesd accelerate pytorch-fid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00


In [2]:
from diffusers import DiffusionPipeline
import torch, tomesd
import pytorch_fid.fid_score as fid
from pytorch_fid.inception import InceptionV3

In [3]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipeline.enable_attention_slicing()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [4]:
image_clean = pipeline("photograph of an astronaut riding a horse", 768, 768, generator=torch.Generator().manual_seed(555035)).images[0]
images_clean = [image_clean]
!mkdir images_clean
image_clean.save("images_clean/image_clean.png")
tomesd.apply_patch(pipeline, ratio=0.2)
image_20 = pipeline("photograph of an astronaut riding a horse", 768, 768, generator=torch.Generator().manual_seed(555035)).images[0]
images_20 = [image_20]
!mkdir images_20
image_20.save("images_20/image_20.png")
#tomesd.remove_patch(pipeline)
tomesd.apply_patch(pipeline, ratio=0.5)
image_50 = pipeline("photograph of an astronaut riding a horse", 768, 768, generator=torch.Generator().manual_seed(555035)).images[0]
images_50 = [image_50]
!mkdir images_50
image_50.save("images_50/image_50.png")
tomesd.remove_patch(pipeline)
print('Done')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Done


In [5]:
'''
import random
n = random.randint(1,1000000)
#print(n)
#tomesd.apply_patch(pipeline, 0.5)
#image = pipeline("photograph of an astronaut riding a horse", 768, 768, 50, generator=torch.Generator().manual_seed(n)).images[0]
image = pipeline("tiger", 768, 768, generator=torch.Generator().manual_seed(759377)).images[0]
#tomesd.remove_patch(pipeline)
!mkdir test
image.save("test/tiger.png")
# tiger: 759377
'''

'\nimport random\nn = random.randint(1,1000000)\n#print(n)\n#tomesd.apply_patch(pipeline, 0.5)\n#image = pipeline("photograph of an astronaut riding a horse", 768, 768, 50, generator=torch.Generator().manual_seed(n)).images[0]\nimage = pipeline("tiger", 768, 768, generator=torch.Generator().manual_seed(759377)).images[0]\n#tomesd.remove_patch(pipeline)\n!mkdir test\nimage.save("test/tiger.png")\n# tiger: 759377\n'

In [6]:
!python3 -m pytorch_fid /content/images_clean/ /content/images_20/

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 224MB/s]
100% 1/1 [00:00<00:00,  1.81it/s]
100% 1/1 [00:00<00:00,  8.51it/s]
FID:  53.83661146914008


In [8]:
dims = 2048
device = torch.device('cuda' if (torch.cuda.is_available()) else 'cpu')
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]
model = InceptionV3([block_idx]).to(device)
batch_size = 1
m1, s1 = fid.calculate_activation_statistics(images_clean, model, batch_size, dims, device, 1)
m2, s2 = fid.calculate_activation_statistics(images_20, model, batch_size, dims, device, 1)
fid_score = fid.calculate_frechet_distance(m1, s1, m2, s2)
fid_score

  0%|          | 0/1 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 6>:6                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pytorch_fid/fid_score.py:228 in                          │
│ calculate_activation_statistics                                                                  │
│                                                                                                  │
│   225 │   -- sigma : The covariance matrix of the activations of the pool_3 layer of             │
│   226 │   │   │      the inception model.                                                        │
│   227 │   """                                                                                    │
│ ❱ 228 │   act = get_activations(files, model, batch_size, dims, device, num_workers)             │
│   229 │   mu = np.mean(act, axis=0)                                                              │
│   230 │   sigma = np.cov(act, rowvar=False)                                                      │
│   231 │   return mu, sigma                                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pytorch_fid/fid_score.py:132 in get_activations          │
│                                                                                                  │
│   129 │                                                                                          │
│   130 │   start_idx = 0                                                                          │
│   131 │                                                                                          │
│ ❱ 132 │   for batch in tqdm(dataloader):                                                         │
│   133 │   │   batch = batch.to(device)                                                           │
│   134 │   │                                                                                      │
│   135 │   │   with torch.no_grad():                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/tqdm/std.py:1178 in __iter__                             │
│                                                                                                  │
│   1175 │   │   time = self._time                                                                 │
│   1176 │   │                                                                                     │
│   1177 │   │   try:                                                                              │
│ ❱ 1178 │   │   │   for obj in iterable:                                                          │
│   1179 │   │   │   │   yield obj                                                                 │
│   1180 │   │   │   │   # Update and possibly print the progressbar.                              │
│   1181 │   │   │   │   # Note: does not call self.update(1) for speed optimisation.              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:633 in __next__           │
│                                                                                                  │
│    630 │   │   │   if self._sampler_iter is None:                                                │
│    631 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    632 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  633 │   │   │   data = self._next_data()                